# 🌾 Rice Classification | Part 3: Evaluation & Metrics
**Arborio vs Jasmine — Binary Classification**

**Prerequisite:** Run Parts 1 & 2 first. Ensure `part2_models.pkl` is in your Colab session.

**Covers:** Accuracy · Precision · Recall · F1 · Confusion Matrix · ROC AUC · Misclassification Analysis · Final Test Evaluation

In [ ]:
# ── Cell 1: Imports ─────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle, warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_curve, roc_auc_score
)

sns.set_theme(style='whitegrid', palette='Set2')
CLASS_NAMES = ['Arborio', 'Jasmine']
print('✅ Imports ready')

In [ ]:
# ── Cell 2: Load Models & Data from Part 2 ───────────────────────
with open('part2_models.pkl', 'rb') as f:
    d = pickle.load(f)

X_train, X_val, X_test = d['X_train'], d['X_val'], d['X_test']
y_train, y_val, y_test = d['y_train'], d['y_val'], d['y_test']
models                 = d['models']
feature_columns        = d['feature_columns']

print('Models loaded:', list(models.keys()))
print(f'Val size: {len(y_val)} | Test size: {len(y_test)}')

In [ ]:
# ── Cell 3: Compute All Metrics for Every Model ──────────────────
def full_metrics(model, X, y_true):
    """Returns dict of all classification metrics for a trained model."""
    y_pred = model.predict(X)
    # predict_proba gives probability scores — column 1 = P(Jasmine)
    y_prob = model.predict_proba(X)[:, 1] if hasattr(model, 'predict_proba') else None
    return {
        'y_pred':    y_pred,
        'y_prob':    y_prob,
        'accuracy':  accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall':    recall_score(y_true, y_pred, zero_division=0),
        'f1':        f1_score(y_true, y_pred, zero_division=0),
        'cm':        confusion_matrix(y_true, y_pred),
        'roc_auc':   roc_auc_score(y_true, y_prob) if y_prob is not None else None,
    }

val_metrics = {name: full_metrics(m, X_val, y_val) for name, m in models.items()}
print('✅ Metrics computed for all models')

In [ ]:
# ── Cell 4: Comprehensive Metrics Table ──────────────────────────
rows = []
for name, m in val_metrics.items():
    rows.append({
        'Model':     name,
        'Accuracy':  f"{m['accuracy']*100:.2f}%",
        'Precision': f"{m['precision']*100:.2f}%",
        'Recall':    f"{m['recall']*100:.2f}%",
        'F1':        f"{m['f1']*100:.2f}%",
        'ROC AUC':   f"{m['roc_auc']:.4f}" if m['roc_auc'] else 'N/A',
    })

table = pd.DataFrame(rows).set_index('Model')
print('📊 METRICS TABLE — Validation Set')
print('=' * 70)
print(table.to_string())
print('=' * 70)
print('\nMetric guide:')
print('  Accuracy  : Overall correct predictions')
print('  Precision : Of predicted Jasmine, how many were truly Jasmine')
print('  Recall    : Of actual Jasmine grains, how many did we catch')
print('  F1        : Harmonic mean of Precision and Recall')
print('  ROC AUC   : 0.5=random | 1.0=perfect')

In [ ]:
# ── Cell 5: Confusion Matrices — All Models ──────────────────────
# Diagonal = correct | Off-diagonal = mistakes
# Top-right (FP) = Arborio called Jasmine | Bottom-left (FN) = Jasmine called Arborio

fig, axes = plt.subplots(2, 2, figsize=(14, 11))
axes = axes.flatten()

for idx, (name, m) in enumerate(val_metrics.items()):
    cm      = m['cm']
    cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    annot   = np.array([[f"{cm[i,j]}\n({cm_norm[i,j]*100:.1f}%)" for j in range(2)] for i in range(2)])

    sns.heatmap(cm_norm, annot=annot, fmt='', cmap='Blues',
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
                ax=axes[idx], vmin=0, vmax=1, linewidths=2, linecolor='white', cbar=False)

    TN, FP, FN, TP = cm.ravel()
    axes[idx].set_title(f'{name}\nAcc: {m["accuracy"]*100:.1f}%  TP={TP} TN={TN} FP={FP} FN={FN}',
                        fontsize=10, fontweight='bold')
    axes[idx].set_ylabel('Actual'); axes[idx].set_xlabel('Predicted')

plt.suptitle('Confusion Matrices — All Models (Validation Set)\n'
             'Diagonal = correct | Off-diagonal = mistakes', fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Cell 6: Radar Chart — Multi-Metric Comparison ───────────────
metric_keys   = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']
N      = len(metric_labels)
angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist() + \
         np.linspace(0, 2*np.pi, N, endpoint=False).tolist()[:1]  # Close the polygon

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
colors  = ['#4C72B0', '#DD8452', '#55A868', '#C44E52']

for (name, m), color in zip(val_metrics.items(), colors):
    vals = [m[k] or 0 for k in metric_keys] + [m[metric_keys[0]] or 0]
    ax.plot(angles, vals, 'o-', linewidth=2, label=name, color=color)
    ax.fill(angles, vals, alpha=0.08, color=color)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metric_labels, fontsize=11, fontweight='bold')
ax.set_ylim(0.80, 1.0)
ax.set_yticks([0.82, 0.87, 0.92, 0.97])
ax.set_yticklabels(['82%','87%','92%','97%'], fontsize=9)
ax.set_title('Radar Chart — Model Comparison\nLarger area = stronger overall', fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.35, 1.15), fontsize=9)
plt.tight_layout()
plt.savefig('radar_chart.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Cell 7: ROC Curves ───────────────────────────────────────────
# x-axis: False Positive Rate (false alarms)
# y-axis: True Positive Rate (correct detections)
# Closer to top-left corner = better | Diagonal = random chance

fig, ax = plt.subplots(figsize=(9, 7))
colors  = ['#4C72B0', '#DD8452', '#55A868', '#C44E52']

for (name, m), color in zip(val_metrics.items(), colors):
    if m['y_prob'] is None: continue
    fpr, tpr, _ = roc_curve(y_val, m['y_prob'])
    ax.plot(fpr, tpr, linewidth=2.5, color=color, label=f'{name}  (AUC={m["roc_auc"]:.4f})')

ax.plot([0,1],[0,1], 'k--', linewidth=1.2, label='Random Chance (AUC=0.5)')
ax.plot(0, 1, 'g*', markersize=15, label='Perfect Classifier', zorder=5)
ax.annotate('Perfect', xy=(0,1), xytext=(0.05, 0.93), color='green', fontsize=9,
            arrowprops=dict(arrowstyle='->', color='green'))

ax.set_xlabel('False Positive Rate (FPR) → false alarms', fontsize=11)
ax.set_ylabel('True Positive Rate (TPR / Recall)', fontsize=11)
ax.set_title('ROC Curves — All Models | Higher & further left = better', fontweight='bold')
ax.legend(loc='lower right', fontsize=9)
ax.set_xlim([-0.01, 1.01]); ax.set_ylim([-0.01, 1.05])
plt.tight_layout()
plt.savefig('roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Cell 8: Which Metric Matters Most? ───────────────────────────
print("""
PRIMARY METRIC: F1 Score

In rice classification both error types have similar costs:
  FP: Arborio labelled as Jasmine → wrong product delivered
  FN: Jasmine labelled as Arborio → premium rice undervalued

When neither error is much worse than the other, F1 is the
right metric — it balances Precision and Recall equally.

Precision vs Recall trade-off:
  Lower threshold → more Jasmine predictions → higher Recall, lower Precision
  Higher threshold → fewer Jasmine predictions → higher Precision, lower Recall
  This trade-off is captured in the ROC curve above.

ROC AUC is a good secondary metric — it is threshold-independent
and measures overall ranking ability (0.5=random, 1.0=perfect).
""")

In [ ]:
# ── Cell 9: Misclassification Analysis ───────────────────────────
# Find the best model by F1 and examine where it goes wrong

best_name    = max(val_metrics, key=lambda k: val_metrics[k]['f1'])
best_m       = val_metrics[best_name]
y_pred_best  = best_m['y_pred']

print(f'Analysing misclassifications for best model: {best_name}')
print(f'Val F1: {best_m["f1"]*100:.2f}%\n')

# Build a DataFrame tagging each sample as correct or an error type
error_df = pd.DataFrame(X_val, columns=feature_columns)
error_df['True']      = ['Arborio' if l==0 else 'Jasmine' for l in y_val]
error_df['Predicted'] = ['Arborio' if l==0 else 'Jasmine' for l in y_pred_best]
error_df['Correct']   = (y_val == y_pred_best)
error_df['Error']     = error_df.apply(
    lambda r: 'FP (called Jasmine, was Arborio)' if r['True']=='Arborio' and r['Predicted']=='Jasmine'
    else ('FN (called Arborio, was Jasmine)' if r['True']=='Jasmine' and r['Predicted']=='Arborio'
    else 'Correct'), axis=1)

errors   = error_df[~error_df['Correct']]
correct  = error_df[error_df['Correct']]

print(f'Correct  : {len(correct)} ({len(correct)/len(y_val)*100:.1f}%)')
print(f'Errors   : {len(errors)} ({len(errors)/len(y_val)*100:.1f}%)')
print(f'\nError breakdown:\n{errors["Error"].value_counts().to_string()}')

print('\nMean feature values — Errors vs Correct:')
comp = pd.DataFrame({
    'Correct':      correct[feature_columns].mean(),
    'Misclassified': errors[feature_columns].mean()
})
comp['Δ%'] = ((comp['Misclassified'] - comp['Correct']) / comp['Correct'] * 100).round(2)
print(comp.round(2).to_string())

In [ ]:
# ── Cell 10: Misclassification Map ───────────────────────────────
# Plot errors in the space of the 2 most discriminative features
# Errors typically cluster near the decision boundary

diff = abs(error_df[error_df['True']=='Arborio'][feature_columns].mean() -
           error_df[error_df['True']=='Jasmine'][feature_columns].mean())
top2 = diff.nlargest(2).index.tolist()

fig, ax = plt.subplots(figsize=(11, 7))

# Background: all correct samples
for label, color in [('Arborio','#6baed6'), ('Jasmine','#fdae6b')]:
    mask = correct['True'] == label
    ax.scatter(correct.loc[mask, top2[0]], correct.loc[mask, top2[1]],
               c=color, alpha=0.3, s=20, label=f'{label} (correct)')

# Foreground: errors
for err_type, marker, color in [
    ('FP (called Jasmine, was Arborio)', 'X', 'red'),
    ('FN (called Arborio, was Jasmine)',  'D', 'darkred')
]:
    mask = errors['Error'] == err_type
    if mask.sum() > 0:
        ax.scatter(errors.loc[mask, top2[0]], errors.loc[mask, top2[1]],
                   c=color, marker=marker, s=120, edgecolors='black',
                   zorder=5, label=f'ERROR: {err_type} (n={mask.sum()})')

ax.set_xlabel(top2[0], fontsize=11); ax.set_ylabel(top2[1], fontsize=11)
ax.set_title(f'Misclassification Map — {best_name}\n'
             'Errors cluster near the class boundary — the hardest samples to separate',
             fontweight='bold')
ax.legend(fontsize=9)
plt.tight_layout()
plt.savefig('misclassification_map.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Cell 11: Final Test Set Evaluation ───────────────────────────
# Best model selected by val F1 — test set evaluated ONCE, here, at the end

best_model_obj = models[best_name]
test_m         = full_metrics(best_model_obj, X_test, y_test)

print(f'🏆 Best model: {best_name}')
print('=' * 56)
print(f'{"Metric":<15} {"Validation":>12} {"Test":>12} {"Δ":>10}')
print('-' * 56)

for key, label in [('accuracy','Accuracy'),('precision','Precision'),
                    ('recall','Recall'),('f1','F1'),('roc_auc','ROC AUC')]:
    v = best_m[key] or 0
    t = test_m[key] or 0
    d = t - v
    direction = '↑' if d > 0 else ('↓' if d < 0 else '=')
    print(f'{label:<15} {v*100:>11.2f}% {t*100:>11.2f}%  {direction}{abs(d)*100:>5.2f}%')

print('=' * 56)

In [ ]:
# ── Cell 12: Val vs Test Confusion Matrices ───────────────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

for ax, (m, title, n) in zip(axes, [
    (best_m,  f'Validation Set ({len(y_val)} samples)',    len(y_val)),
    (test_m,  f'Test Set — Final Exam ({len(y_test)} samples)', len(y_test))
]):
    cm      = m['cm']
    cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    annot   = np.array([[f"{cm[i,j]}\n({cm_norm[i,j]*100:.1f}%)" for j in range(2)] for i in range(2)])
    sns.heatmap(cm_norm, annot=annot, fmt='', cmap='Greens',
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
                ax=ax, vmin=0, vmax=1, linewidths=2, linecolor='white', annot_kws={'size':12})
    ax.set_title(f'{best_name}\n{title}\nAcc: {m["accuracy"]*100:.2f}%  F1: {m["f1"]*100:.2f}%',
                 fontweight='bold', fontsize=10)
    ax.set_xlabel('Predicted'); ax.set_ylabel('Actual')

plt.suptitle('🏆 Best Model: Validation vs Test Performance', fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('final_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Cell 13: Generalisation Verdict ──────────────────────────────
gap = abs(best_m['accuracy'] - test_m['accuracy'])

if   gap < 0.01: verdict = '✅ EXCELLENT — virtually no drop on unseen test data'
elif gap < 0.03: verdict = '✅ GOOD — small acceptable drop'
elif gap < 0.05: verdict = '⚠️  MODERATE — consider more regularisation'
else:            verdict = '❌ POOR — large gap suggests overfitting'

print('GENERALISATION ANALYSIS')
print('=' * 45)
print(f'  Val Accuracy  : {best_m["accuracy"]*100:.2f}%')
print(f'  Test Accuracy : {test_m["accuracy"]*100:.2f}%')
print(f'  Gap           : {gap*100:.2f}%')
print(f'  Verdict       : {verdict}')
print('=' * 45)

print('\n✅ All three parts complete!')
print('   Outputs saved: confusion_matrices.png | radar_chart.png | roc_curves.png')
print('                  misclassification_map.png | final_evaluation.png')